# Семинар 9: KNN және SVM-ді практикалық қолдану

**Семинар мақсаты:** K-ең жақын көршілер (KNN) және Тірек векторлар (SVM) әдістері арқылы бинарлық классификация есебін шешудің толық циклін іс жүзінде, қадам-қадаммен талдау. Біз `scikit-learn` кітапханасын қолданамыз.

**Мәселе:** Сонар деректері бойынша суасты нысандарын жіктеу. Бізге әртүрлі жиіліктерде сонардан алынған 60 сигнал негізінде нысанның **мина (`M`)** немесе **тас (`R`)** екенін анықтау керек.

**Неліктен бұл деректер жинағы таңдалды?**
*   Бұл күрделі бинарлық классификация есебі.
*   Белгілер көп (60), сондықтан шешімді "көру" үшін деректерді жай ғана визуализациялай алмаймыз.
*   Деректер сызықтық түрде бөлінбейді, бұл бізге SVM-дегі сызықтық емес ядролардың күшін көруге мүмкіндік береді.

### 1-қадам: Деректерді жүктеу және бастапқы талдау

**Не істейміз:** Файлдан деректерді жүктеп, олардың құрылымына қараймыз. Бос орындардың жоқтығына көз жеткіземіз және белгілеріміз бен мақсатымыздың түрін түсінеміз.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Деректерді жүктейміз. Бұл файлда тақырыптар жоқ, сондықтан header=None.
# Соңғы, 61-ші баған (индекс 60) - бұл біздің мақсатымыз.
df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/sonar.csv', header=None)

# Алғашқы 5 жолды қарастырайық
print("Деректер жинағының алғашқы 5 жолы:")
display(df.head())

# Жалпы ақпаратты қарап, бос орындарды тексерейік
print("\nДеректер жинағы туралы ақпарат:")
df.info()

# Кластардың балансын қарастырайық
print("\nКластардың балансы (M - мина, R - тас):")
print(df[60].value_counts())

**Бірінші қадамнан шыққан қорытындылар:**
*   Деректер сәтті жүктелді. Бізде 208 нысан және 61 баған бар.
*   Барлық 60 белгінің сандық түрі бар (`float64`).
*   Мақсатты айнымалы (60-баған) `object` (жол) түріне ие.
*   Деректерде бос орындар жоқ.
*   Кластар жеткілікті түрде теңгерімді (111 Мина және 97 Тас), бұл оқыту үшін жақсы.

### 2-қадам: Оқытуға арналған деректерді дайындау

**Не істейміз:** Деректерімізді `X` белгілер матрицасына және `y` мақсатты айнымалы векторына бөлеміз. Содан кейін оларды оқыту және тест жиынтықтарына бөлеміз. Бұл модельді объективті бағалау үшін **негізгі** қадам.

In [ ]:
from sklearn.model_selection import train_test_split

# X - бұл соңғы бағаннан басқа барлық бағандар (белгілер)
X = df.drop(60, axis=1)

# y - бұл соңғы баған (мақсат)
y = df[60]

# Деректерді оқыту (75%) және тест (25%) жиынтықтарына бөлеміз
# random_state=42 бөлудің әрқашан бірдей болатынына кепілдік береді,
# бұл нәтижелерді қайталауға мүмкіндік береді.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### 3-қадам: KNN моделін құру және оқыту

**Не істейміз:** Біз `Pipeline` және `GridSearchCV` қолданамыз.
*   `Pipeline` — бұл бірнеше өңдеу қадамдарын біріктіруге мүмкіндік беретін конвейер. Біздің жағдайда бұл 'деректерді масштабтау' -> 'KNN моделі' болады. Бұл өте ыңғайлы және тест жиынтығынан деректердің ағып кетуіне жол бермейді.
*   `GridSearchCV` — ең жақсы гиперпараметрлерді автоматты түрде таңдауға арналған құрал. Біз оған `k` (көршілер саны) мәндерінің диапазонын береміз, ал ол оңтайлысын табады.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# 3.1-қадам: Конвейер (Pipeline) құрамыз
# Бірінші қадам - 'scaler': деректерді стандарттау (орташаны 0-ге, дисперсияны 1-ге келтіру)
# Екінші қадам - 'knn': біздің модель
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 3.2-қадам: Іздеуге арналған параметрлер торын анықтаймыз
# Біз GridSearchCV-дің 1-ден 29-ға дейінгі барлық k мәндерін тексергенін қалаймыз.
# 'knn__n_neighbors' параметрінің атауы Pipeline-дегі қадам атауынан ('knn'),
# екі астын сызудан '__' және модельдегі параметр атауынан ('n_neighbors') тұрады.
param_grid_knn = {'knn__n_neighbors': range(1, 30)}

# 3.3-қадам: GridSearchCV-ді құрып, оқытамыз
# cv=5 әр параметрді бағалау үшін 5 еселі кросс-валидацияны білдіреді.
grid_knn = GridSearchCV(knn_pipe, param_grid_knn, cv=5, scoring='accuracy')

# Оқыту және параметрлерді таңдау процесін іске қосамыз
grid_knn.fit(X_train, y_train)

# Табылған ең жақсы параметрді шығарамыз
print(f"KNN үшін ең жақсы K мәні: {grid_knn.best_params_['knn__n_neighbors']}")

### 4-қадам: KNN моделінің сапасын бағалау

**Не істейміз:** `GridSearchCV` тапқан ең жақсы модельді пайдалана отырып, тест деректерінде болжамдар жасаймыз және олардың сапасын қателіктер матрицасы мен жіктеу есебі арқылы бағалаймыз.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Тест жиынтығында болжамдар жасаймыз
y_pred_knn = grid_knn.predict(X_test)

print("\n--- KNN моделінің сапасы бойынша есеп ---")
print(classification_report(y_test, y_pred_knn))

print("KNN үшін қателіктер матрицасы:")
print(confusion_matrix(y_test, y_pred_knn))

**KNN нәтижелерін түсіндіру:**
*   `classification_report` бізге әр класс үшін `precision`, `recall` және `f1-score` мәндерін көрсетеді.
*   `accuracy` (жалпы дәлдік) дұрыс жауаптардың үлесін көрсетеді.
*   Қателіктер матрицасы модельдің қай кластарды шатастыратынын көрсетеді. Бұл жағдайда k=1 болатын модель 5 тасты (оларды мина деп болжады) және 1 минаны (тас деп болжады) қателесті.

### 5-қадам: SVM моделін құру, оқыту және бағалау

**Не істейміз:** Дәл сол процесті қайталаймыз, бірақ енді SVM моделі үшін. Мұнда біз басқа гиперпараметрлерді таңдаймыз: `C` (қателік үшін айыппұл) және `gamma` (бір мысалдың әсері).

In [ ]:
from sklearn.svm import SVC

# 5.1-қадам: SVM үшін Pipeline құрамыз
svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 5.2-қадам: SVM үшін параметрлер торын анықтаймыз
# Біз C және gamma комбинацияларын тексереміз. Ядроны 'rbf' - ең әмбебап түрінде қалдырамыз.
param_grid_svm = {
    'svm__C': [0.1, 1, 10, 50, 100],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01, 0.001]
}

# 5.3-қадам: SVM үшін GridSearchCV құрып, оқытамыз
grid_svm = GridSearchCV(svm_pipe, param_grid_svm, cv=5, scoring='accuracy')
grid_svm.fit(X_train, y_train)

# Ең жақсы параметрлерді шығарамыз
print(f"SVM үшін ең жақсы параметрлер: {grid_svm.best_params_}")

# 5.4-қадам: Ең жақсы SVM моделін бағалау
y_pred_svm = grid_svm.predict(X_test)
print("\n--- SVM моделінің сапасы бойынша есеп ---")
print(classification_report(y_test, y_pred_svm))

print("SVM үшін қателіктер матрицасы:")
print(confusion_matrix(y_test, y_pred_svm))

### 6-қадам: Модельдерді қорытынды салыстыру

**Не істейміз:** Нәтижелерді бір кестеге жинаймыз немесе қай модельдің осы тапсырманы жақсы орындағаны туралы қорытынды жасау үшін есептерді салыстырамыз.

**Нәтижелер:**

| Метрика | KNN (k=1) | SVM (C=10, gamma='scale') |
| :--- | :--- | :--- |
| **Accuracy** | 0.90 | **0.94** |
| **F1-score (M)** | 0.92 | **0.95** |
| **F1-score (R)** | 0.89 | **0.93** |

**Семинардың жалпы қорытындысы:**

1.  Біз екі түрлі модель үшін ML-дің толық циклін сәтті жүзеге асырдық: деректерді жүктеуден бастап сапаны бағалауға дейін.
2.  Біз гиперпараметрлерді таңдаудың қаншалықты маңызды екенін және `GridSearchCV` бұл процесті қалай автоматтандыратынын көрдік.
3.  Деректерді дұрыс масштабтау үшін `Pipeline` қолдану қажеттілігіне көз жеткіздік.
4.  Осы нақты тапсырмада, гиперпараметрлерді таңдағаннан кейін, **SVM моделі** барлық негізгі метрикалар бойынша KNN-ге қарағанда **сәл жақсырақ нәтиже көрсетті**. Бұл кластар арасындағы шекара күрделі және сызықтық емес болған жағдайларда жиі кездеседі.